# Week3  Session based recommendations using RNN

REFERENCE: https://github.com/mquad/sars_tutorial/blob/master/06_PersonalizedRNN.ipynb

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
from util.data_utils import create_seq_db_filter_top_k, sequences_to_spfm_format
from util.split import last_session_out_split
from util.metrics import precision, recall, mrr
from util import evaluation
from recommenders.RNNRecommender import RNNRecommender

ModuleNotFoundError: No module named 'util'